In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn

In [2]:
df=pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\code\datasets\drug200.csv")
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


In [3]:
df[df.isna().any(axis=1)]

,Age,Sex,BP,Cholesterol,Na_to_K,Drug


In [5]:
df.describe(include='all')

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
count,200.000000,200,200,200,200.000000,200
unique,NaN,2,3,2,NaN,5
top,NaN,M,HIGH,HIGH,NaN,DrugY
freq,NaN,104,77,103,NaN,91
mean,44.315000,NaN,NaN,NaN,16.084485,NaN
std,16.544315,NaN,NaN,NaN,7.223956,NaN
min,15.000000,NaN,NaN,NaN,6.269000,NaN
25%,31.000000,NaN,NaN,NaN,10.445500,NaN
50%,45.000000,NaN,NaN,NaN,13.936500,NaN
75%,58.000000,NaN,NaN,NaN,19.380000,NaN


In [16]:
a=set(df['Drug'])
b=set(df['BP'])
c=set(df['Cholesterol'])
print(a,b,c)

{'drugC', 'DrugY', 'drugA', 'drugB', 'drugX'} {'LOW', 'NORMAL', 'HIGH'} {'NORMAL', 'HIGH'}


In [23]:
from pandas import get_dummies
dfv2=get_dummies(df,columns=['Sex','BP','Cholesterol','Drug'],dtype="float")


In [24]:
dfv2.head()

,Age,Na_to_K,Sex_F,Sex_M,BP_HIGH,BP_LOW,BP_NORMAL,Cholesterol_HIGH,Cholesterol_NORMAL,Drug_DrugY,Drug_drugA,Drug_drugB,Drug_drugC,Drug_drugX
0,23,25.355,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,47,13.093,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,47,10.114,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,28,7.798,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,61,18.043,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [27]:
col=list(dfv2.columns)
ycol=[]
[ycol.append(x) for x in col if x.startswith('Drug')]
[col.remove(x) for x in ycol]

[None, None, None, None, None]

In [30]:
X=dfv2[col]
Y=dfv2[ycol]

xarray=np.array(X)
yarray=np.array(Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(xarray, yarray, test_size=0.2, random_state=42)
X_val,X_test,y_val,y_test=train_test_split(X_test,y_test, test_size=0.5,random_state=42)

In [36]:
print(X_train.shape,y_train.shape)

(160, 9) (160, 5)


In [42]:
class dataset(Dataset):
    def __init__(self,A,B):
        self.X=torch.tensor(A,dtype=torch.float32)
        self.Y=torch.tensor(B,dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index],self.Y[index]

In [43]:
traindataset=dataset(A=X_train,B=y_train)
valdataset=dataset(A=X_val,B=y_val)
testdataset=dataset(A=X_test,B=y_test)

traindata_laoder= DataLoader(traindataset, batch_size= 8, shuffle=True)
testdata_loader= DataLoader(testdataset, batch_size= 8,shuffle=True)
valdata_loader=DataLoader(valdataset,batch_size= 8,shuffle=True)

In [48]:
class Classification(nn.Module):
    def __init__(self):
        super(Classification,self).__init__()
        self.Linear1=nn.Linear(X.shape[1],10)
        self.relu=nn.ReLU()
        self.Linear2=nn.Linear(10,Y.shape[1])

    def forward(self,x):
        x=self.Linear1(x)
        x=self.relu(x)
        x=self.Linear2(x)

model=Classification()

In [53]:
from torchsummary import summary

summary(model,input_size=(xarray.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             100
              ReLU-2                   [-1, 10]               0
            Linear-3                    [-1, 5]              55
Total params: 155
Trainable params: 155
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [ ]:
for i in range(10):
    for value,label in traindata_laoder:
        output=model.compile()


tensor([[52.0000,  9.8940,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,  1.0000,
          0.0000],
        [49.0000, 10.5370,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,
          0.0000],
        [61.0000, 18.0430,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,
          0.0000],
        [58.0000, 14.2390,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
          1.0000],
        [70.0000, 13.9670,  0.0000,  1.0000,  1.0000,  0.0000,  0.0000,  1.0000,
          0.0000],
        [39.0000, 13.9380,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          1.0000],
        [37.0000, 23.0910,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
          1.0000],
        [31.0000, 30.3660,  0.0000,  1.0000,  1.0000,  0.0000,  0.0000,  1.0000,
          0.0000]]) tensor([[0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0.